# Sparse reproduction of clustering method

In [1]:
import csv
import math
import networkx as nx
from operator import itemgetter
from datetime import datetime, timedelta
from random import sample, choice
from statistics import mean, median_low
from collections import Counter, defaultdict
from tqdm.notebook import tqdm
from fog.tokenizers import WordTokenizer
from fog.metrics import sparse_normalize, sparse_dot_product
from fog.evaluation import best_matching_macro_average, clusters_to_labels
from twitwi.constants import TWEET_DATETIME_FORMAT
from stop_words import STOP_WORDS_FR
from typing import List, Dict
from ebbe import distinct, partitioned_items
from sklearn.metrics import fowlkes_mallows_score, v_measure_score, homogeneity_score, completeness_score

In [2]:
%load_ext Cython

## Constants and helpers

In [3]:
ONE_DAY = timedelta(days=1)

def parse_date(created_at):
    return datetime.strptime(created_at, TWEET_DATETIME_FORMAT)

## Reading tweets file

In [4]:
with open('../data/event2018.tsv') as f:
    TWEETS = sorted(
        distinct(csv.DictReader(f, delimiter='\t'), key=itemgetter('id')),
        key=itemgetter('id')
    )

In [5]:
TWEETS = [t for t in TWEETS if t['label']]

In [6]:
# Adding dates & parsing stuff
for i, tweet in enumerate(TWEETS):
    tweet['index'] = i
    tweet['event'] = int(tweet['event'])
    tweet['date'] = parse_date(tweet['created_at'])
    tweet['timestamp'] = tweet['date'].timestamp()
    tweet['label'] = int(tweet['label'].split('.')[0]) if tweet['label'] else None

In [7]:
TWEETS_INDEX = {t['id']: t for t in TWEETS}

In [8]:
days = Counter(t['date'].isoformat()[:10] for t in TWEETS)
WINDOW = int(mean(days.values()))
WINDOW

4354

In [9]:
for k, v in TWEETS[0].items():
    print(k, v)

id 1018722125941755905
label_day 0.0
event 20180716001
text #Rennes - La sortie de prison de Djamel Beghal [Vidéo exclusive] via @letelegramme https://t.co/tbOthY1Ren
text+quote+reply #Rennes - La sortie de prison de Djamel Beghal [Vidéo exclusive] via @letelegramme https://t.co/tbOthY1Ren  
image 
url_image 
user1 True
user2 True
user3 True
created_at Mon Jul 16 05:00:56 +0000 2018
label 0
index 0
date 2018-07-16 05:00:56
timestamp 1531710056.0


In [10]:
print('Total number of tweets:', len(TWEETS))
print('Total number of events:', len(set(t['event'] for t in TWEETS)))
print('Total number of labels:', len(set(t['label'] for t in TWEETS if t['label'] is not None)))
print('Total number of tweets not labeled', sum(1 if t['label'] is None else 0 for t in TWEETS))

Total number of tweets: 95796
Total number of events: 327
Total number of labels: 257
Total number of tweets not labeled 0


In [11]:
TRUTH = partitioned_items((t['label'], t['id']) for t in TWEETS if t['label'] is not None)

In [12]:
def print_cluster_stats(clusters):
    lens = [len(cluster) for cluster in clusters]
    
    print('Number of clusters:', len(clusters))
    print('Number of non-singleton clusters:', sum(l > 1 for l in lens))
    print('Max number of tweets in clusters:', max(lens))
    print('Min number of tweets in clusters:', min(lens))
    print('Mean number of tweets in clusters:', mean(lens))
    print('Median number of tweets in clusters:', median_low(lens))
    print('Median number of tweets in non-singleton clusters', median_low(l for l in lens if l > 1))

In [13]:
print_cluster_stats(TRUTH)

Number of clusters: 257
Number of non-singleton clusters: 257
Max number of tweets in clusters: 18020
Min number of tweets in clusters: 2
Mean number of tweets in clusters: 372.74708171206225
Median number of tweets in clusters: 76
Median number of tweets in non-singleton clusters 76


## Tokenization & Vectorization

*NOTE: It might be useful to convert tokens to incremental ids to speed up hash computations*

In [14]:
tokenizer = WordTokenizer(
    keep=['word'],
    lower=True,
    unidecode=True,
    split_hashtags=True,
    stoplist=STOP_WORDS_FR + [t + "'" for t in STOP_WORDS_FR] + [t + "’" for t in STOP_WORDS_FR],
    reduce_words=True,
    decode_html_entities=True
)

In [15]:
sample_to_tokenize = sample(TWEETS, 5)

for tweet in sample_to_tokenize:
    print(tweet['text'])
    print(list(tokenizer(tweet['text'])))
    print()

New post: "Puigdemont y las falsas expectativas" https://t.co/SkPNH1RLkI
[('word', 'new'), ('word', 'post'), ('word', 'puigdemont'), ('word', 'las'), ('word', 'falsas'), ('word', 'expectativas')]

Avron -&gt; « Nous Avron Gagné » CDG – Etoile -&gt;  « On a 2 Étoiles » Victor Hugo -&gt; « Victor Hugo Lloris » Bercy -&gt; « Bercy les Bleus » Notre-Dame des Champs -&gt;« Notre Didier Deschamps » Champs Elysées-Clémenceau -&gt; « Deschamps Elysées – Clémenceau » #RATP
[('word', 'avron'), ('word', 'avron'), ('word', 'gagne'), ('word', 'cdg'), ('word', 'etoile'), ('word', 'etoiles'), ('word', 'victor'), ('word', 'hugo'), ('word', 'victor'), ('word', 'hugo'), ('word', 'lloris'), ('word', 'bercy'), ('word', 'bercy'), ('word', 'bleus'), ('word', 'notre-dame'), ('word', 'champs'), ('word', 'didier'), ('word', 'deschamps'), ('word', 'champs'), ('word', 'elysees-clemenceau'), ('word', 'deschamps'), ('word', 'elysees'), ('word', 'clemenceau'), ('word', 'ratp')]

#affairebenalla Ici Londres.. Ici Lo

In [16]:
DOCUMENT_FREQUENCIES = Counter()

for tweet in tqdm(TWEETS):
    tweet['tokens'] = set(token for _, token in tokenizer(tweet['text']))
    for token in tweet['tokens']:
        DOCUMENT_FREQUENCIES[token] += 1

  0%|          | 0/95796 [00:00<?, ?it/s]

In [17]:
print('Size of vocabulary:', len(DOCUMENT_FREQUENCIES))

Size of vocabulary: 57091


In [18]:
N = len(TWEETS)
TOKEN_IDS = {}
INVERSE_DOCUMENT_FREQUENCIES = {}

for i, (token, df) in enumerate(DOCUMENT_FREQUENCIES.items()):
    if df > 10:
        TOKEN_IDS[token] = i
        INVERSE_DOCUMENT_FREQUENCIES[token] = 1 + math.log((N + 1) / (df + 1))

In [19]:
print('Size of vocabulary after df trimming:', len(INVERSE_DOCUMENT_FREQUENCIES))

Size of vocabulary after df trimming: 9704


In [20]:
VECTORS: List[Dict[int, float]] = []

for i, tweet in tqdm(enumerate(TWEETS), total=len(TWEETS)):
    vector = {}

    for token in tweet['tokens']:
        idf = INVERSE_DOCUMENT_FREQUENCIES.get(token)
        
        if idf is None:
            continue
        
        vector[TOKEN_IDS[token]] = idf
        
    # TODO: I need to make fog's sparse_normalize mutating
    vector = sparse_normalize(vector)
    VECTORS.append(vector)
    TWEETS[i]['vector'] = vector

  0%|          | 0/95796 [00:00<?, ?it/s]

In [21]:
VECTORS[254]

{541: 0.2688992359918619,
 542: 0.2774942754803345,
 68: 0.2774942754803345,
 85: 0.2747254653925353,
 543: 0.321455114985555,
 544: 0.28384648546562263,
 174: 0.2498417554353074,
 545: 0.19237199497632934,
 546: 0.18464291051057577,
 78: 0.2758052159443864,
 547: 0.323727011709852,
 548: 0.23632412414120496,
 549: 0.25353371544288056,
 550: 0.21442209835940496,
 551: 0.18389370786038023}

In [22]:
sum(bool(v) for v in VECTORS) / len(VECTORS)

0.9967952732890726

## Clustering

In [23]:
%%cython
from collections import deque, defaultdict
from statistics import mean, median

def dot_product(A: dict, B: dict):
    
    # Swapping so we iterate over the smallest set
    if len(A) > len(B):
        A, B = B, A

    cdef float product = 0.0

    for k, w1 in A.items():
        w2 = B.get(k)

        if w2 is not None:
            product += w1 * w2

    return 1.0 - product

def clustering(tweets, window, threshold=0.7):
    best_candidate = None
    cdef float best_distance
    cdef int thread_id = -1
    
    threads = {}
    
    T = deque()
    I = defaultdict(deque)
    M = []
    
    for t1 in tweets:
        
        best_candidate = None
        best_distance = 2.0
        
        # TODO: index current tweet in sparse map
        # TODO: it's possible to compute the cosine online when fetching candidates
        
        C = set()
        
        g = 0
        for dim, _ in sorted(t1['vector'].items(), key=lambda x: (x[1], x[0]), reverse=True):
            
            if g < 256:
                for c in I[dim]:
                    C.add(c)
            g += 1
            I[dim].append(t1['index'])
            
        M.append(len(C))
        
        for c in C:
            t2 = tweets[c]
            d = dot_product(t1['vector'], t2['vector'])
            
            if d > threshold:
                continue
            
            if d < best_distance:
                best_distance = d
                best_candidate = t2

        if best_candidate is not None:
            threads[t1['index']] = threads[best_candidate['index']]
        else:
            thread_id += 1
            threads[t1['index']] = thread_id
            
        yield (t1['index'], threads[t1['index']])

        T.append(t1)

        if len(T) > window:
            t3 = T.popleft()
            
            for dim in t3['vector'].keys():
                I[dim].popleft()
                
    print(mean(M), median(M))

In [24]:
threads = []

for i, thread_id in tqdm(clustering(TWEETS, WINDOW, 0.7), total=len(TWEETS)):
    threads.append((i, thread_id))

  0%|          | 0/95796 [00:00<?, ?it/s]

581.371153284062 399.0


In [25]:
CLUSTERS = partitioned_items((thread_id, TWEETS[i]['id']) for i, thread_id in threads)

## Evaluation

In [26]:
truth_ids = set()
truth_labels = {}
truth_order = {}

c = 0
for i, cluster in enumerate(TRUTH):
    for _id in cluster:
        truth_ids.add(_id)
        truth_labels[_id] = i
        truth_order[_id] = TWEETS_INDEX[_id]['index']
        c += 1

predicted_labels = {}
for i, cluster in enumerate(CLUSTERS):
    for _id in cluster:
        if _id not in truth_labels:
            continue
            
        predicted_labels[_id] = i

truth_labels = [v for _, v in sorted(truth_labels.items(), key=lambda t: truth_order[t[0]])]
predicted_labels = [v for _, v in sorted(predicted_labels.items(), key=lambda t: truth_order[t[0]])]

In [27]:
print_cluster_stats(TRUTH)

Number of clusters: 257
Number of non-singleton clusters: 257
Max number of tweets in clusters: 18020
Min number of tweets in clusters: 2
Mean number of tweets in clusters: 372.74708171206225
Median number of tweets in clusters: 76
Median number of tweets in non-singleton clusters 76


In [28]:
print_cluster_stats(CLUSTERS)

Number of clusters: 14398
Number of non-singleton clusters: 2557
Max number of tweets in clusters: 4859
Min number of tweets in clusters: 1
Mean number of tweets in clusters: 6.653424086678705
Median number of tweets in clusters: 1
Median number of tweets in non-singleton clusters 3


In [29]:
# Best matching macro average
best_matching_macro_average(TRUTH, CLUSTERS, allow_additional_items=True)

(0.8798930674656514, 0.7099192593075736, 0.7363080016499431)

In [30]:
# Fowlkes-Mallows score
fowlkes_mallows_score(truth_labels, predicted_labels)

0.22241649983323722

In [31]:
# Homogeneity score
homogeneity_score(truth_labels, predicted_labels)

0.8767583991120129

In [32]:
# Completeness score
completeness_score(truth_labels, predicted_labels)

0.5490804651924169

In [33]:
# v-measure
v_measure_score(truth_labels, predicted_labels)

0.6752669206848005

## Using original metric

In [34]:
import pandas as pd
import time

In [35]:
data_df = pd.DataFrame(TWEETS).sort_values("id").reset_index(drop=True)

In [36]:
def cluster_event_match(data, pred):
    data["pred"] = pd.Series(pred, dtype=data.label.dtype)

    data = data[data.label.notna()]
    print("{} labels, {} preds".format(len(data.label.unique()), len(data.pred.unique())))
    t0 = time.time()

    match = data.groupby(["label", "pred"], sort=False).size().reset_index(name="a")
    b, c = [], []
    for idx, row in match.iterrows():
        b_ = ((data["label"] != row["label"]) & (data["pred"] == row["pred"]))
        b.append(b_.sum())
        c_ = ((data["label"] == row["label"]) & (data["pred"] != row["pred"]))
        c.append(c_.sum())
    print("match clusters with events took: {} seconds".format(time.time() - t0))
    match["b"] = pd.Series(b)
    match["c"] = pd.Series(c)
    # recall = nb true positive / (nb true positive + nb false negative)
    match["r"] = match["a"] / (match["a"] + match["c"])
    # precision = nb true positive / (nb true positive + nb false positive)
    match["p"] = match["a"] / (match["a"] + match["b"])
    match["f1"] = 2 * match["r"] * match["p"] / (match["r"] + match["p"])
    match = match.sort_values("f1", ascending=False)
    macro_average_f1 = match.drop_duplicates("label").f1.mean()
    macro_average_precision = match.drop_duplicates("label").p.mean()
    macro_average_recall = match.drop_duplicates("label").r.mean()
    return macro_average_precision, macro_average_recall, macro_average_f1

In [37]:
cluster_event_match(data_df, predicted_labels)

257 labels, 14398 preds
match clusters with events took: 25.84272599220276 seconds


(0.8798930674656512, 0.7099192593075742, 0.7363080016499435)